# Intalling necessary webscraping tools

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forg

In [62]:
#importing libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import lxml
import html5lib
import requests
print("Libraries imported.")

Libraries imported.


Use beautifulsoup to get the content of page and then find the table inside the content. After that, put it into a dataframe

In [63]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#Getting content of the page
soup = BeautifulSoup(res.content,'lxml')

#Finding table in the content
table = soup.find_all('table')[0] 

#Put the table into pandas dataframe
df = pd.read_html(str(table))[0]
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [64]:
# Dropping the rows that has 'not assigned' value in both Borough and Neighborhood cells
df_drop = df.drop(df[(df['Borough'] =='Not assigned') & (df['Neighborhood'] == 'Not assigned')].index)

#Reset indexes
df_drop.reset_index(drop=True, inplace=True)

# if a row has a Borough and Neighborhood is 'Not asssigned', change neighborhood to Borough's name.
for i in range(df_drop.shape[0]):
    if df_drop.at[i, 'Neighborhood'] == 'Not assigned':
        df_drop.at[i, 'Neighborhood']=df_drop.at[i, 'Borough']

df_drop.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [65]:
#Neighborhoods that have same postal codes go into the same cell.
df_grouped = df_drop.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [66]:
df_grouped.shape

(103, 3)

# In this part, dataframe will be improved by adding coordinates

In [67]:
#Using geospatial csv file that is provided for coordinates
!wget -q -O 'coordinates.csv' http://cocl.us/Geospatial_data

geo_cord = pd.read_csv('coordinates.csv')
geo_cord.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_cord

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [68]:
#Merging two dataframes

merged_df = pd.merge(df_grouped, geo_cord, on='PostalCode')
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


# Now it is time to explore and cluster neighborhoods in Toronto

In [69]:
#Foursquare credentials

CLIENT_ID = 'Your Foursquare Client ID' # Foursquare ID
CLIENT_SECRET = 'Your Foursquare Secret' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [70]:
#Make a dataframe that has Toronto in its Borough column
toronto_frame = merged_df.loc[merged_df['Borough'].str.contains('Toronto')]
toronto_frame.reset_index(drop=True, inplace=True)
print(toronto_frame.shape)
toronto_frame.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Define a function to get venues

In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
#Let's get venues from boroughs that has 'Toronto' in its name.

toronto_venues = getNearbyVenues(names=toronto_frame['Neighborhood'],
                                   latitudes=toronto_frame['Latitude'],
                                   longitudes=toronto_frame['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [73]:
print("Toronto venues shape: ", toronto_venues.shape)
toronto_venues

Toronto venues shape:  (1670, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
...,...,...,...,...,...,...,...
1665,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1666,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1667,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1668,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio


How many venues for each neighborhood ?

In [74]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",90,90,90,90,90,90
Christie,17,17,17,17,17,17


How many unique categories are there ? 

In [75]:
len(toronto_venues['Venue Category'].unique())

233

# Let's analyze each neighborhood

In [76]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
#put neighborhood column to first column
toronto_onehot=toronto_onehot.reindex(columns = (['Neighborhood'] + list([a for a in toronto_onehot.columns if a !='Neighborhood'])))
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
#Shape of new dataframe
toronto_onehot.shape

(1670, 233)

Let's group by neighborhood and see frequency of each category

In [78]:
toronto_freq = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_freq.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.062500
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.071429,0.071429,0.071429,0.214286,0.142857,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


New shape

In [79]:
toronto_freq.shape

(38, 233)

Print each neighborhood with top 5 most common venues

In [80]:
num_top_venues = 5

for hood in toronto_freq['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_freq[toronto_freq['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.04
3      Beer Bar  0.04
4    Steakhouse  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.05
4    Intersection  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1     Garden Center  0.06
2              Park  0.06
3        Comic Shop  0.06
4  Recording Studio  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.21
1  Airport Terminal  0.14
2   Harbor / Marina  0.07
3           Airport  0.07
4    

# Let's put this into a dataframe

Let's write a function to sort values into descending order

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now create a dataframe and show top 10 venues for each neighborhood

In [127]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_freq['Neighborhood']

for ind in np.arange(toronto_freq.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_freq.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(38, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Bakery,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Grocery Store
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Lounge,Harbor / Marina,Bar,Sculpture Garden,Boutique,Boat or Ferry


## Time to Cluster Neighborhoods

Use k-means clustering.
Cluster neighborhood into 5 clusters.

In [132]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

toronto_clustering = toronto_freq.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       3, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe with cluster labels

In [133]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_frame

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Pub,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Yoga Studio,Diner,Discount Store,Dog Run
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0.0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Diner,Pub,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0.0,Sandwich Place,Park,Brewery,Movie Theater,Pub,Food & Drink Shop,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Board Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Diner,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3.0,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [134]:
#Dropping NaN values
toronto_merged.dropna(axis=0, inplace=True)
#and cluster labels are float. so i need to change them to integer
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(np.int64)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Yoga Studio,Diner,Discount Store,Dog Run
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Diner,Pub,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Park,Brewery,Movie Theater,Pub,Food & Drink Shop,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Board Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Diner,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


# Finally visualize 

In [26]:
# We need folium for map
!conda install -c conda-forge folium=0.5.0 --yes
# We need geopy to get adresses' coordinates
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:   

In [135]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [136]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [137]:
# Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Yoga Studio,Diner,Discount Store,Dog Run
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Diner,Pub,Caribbean Restaurant
2,East Toronto,0,Sandwich Place,Park,Brewery,Movie Theater,Pub,Food & Drink Shop,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Board Shop
3,East Toronto,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Yoga Studio,Diner,Pet Store,Park
5,Central Toronto,0,Gym,Hotel,Clothing Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Travel,Gluten-free Restaurant,Empanada Restaurant
6,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Diner,Mexican Restaurant,Dessert Shop,Park,Clothing Store,Chinese Restaurant,Café,Rental Car Location
7,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Gym,Italian Restaurant,Café,Pharmacy,Seafood Restaurant
9,Central Toronto,0,Coffee Shop,Pub,Bagel Shop,Fried Chicken Joint,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Light Rail Station
11,Downtown Toronto,0,Coffee Shop,Park,Restaurant,Café,Italian Restaurant,Pub,Bakery,Pizza Place,Pet Store,Sandwich Place
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


In [139]:
# Cluster 1 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Gym,Playground,Park,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [140]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [141]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
23,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
